In [86]:
import csv
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
from collections import Counter
from functools import reduce
from keras import backend as K 
K.set_image_dim_ordering('th')
from keras.models import Sequential
from keras import applications
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2
import os.path
from keras.models import Model
from sklearn.preprocessing import MultiLabelBinarizer
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.applications.imagenet_utils import decode_predictions
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [48]:
metadata = pd.read_csv('/mnt/metadata_sf.csv')
metadata.rename(columns={'id':'image_id', 'datetaken':'date_taken'}, inplace=True)

In [49]:
def prepare_image(image_path):
    if os.path.exists(image_path):
#         im = cv2.resize(cv2.imread(image_path), (224, 224)).astype(np.float32)
#         im[:,:,0] -= 103.939
#         im[:,:,1] -= 116.779
#         im[:,:,2] -= 123.68
#         im = im.transpose((2,0,1))
#         im = np.expand_dims(im, axis=0)
#         return im
#     else:
#         return None
        img = load_img(image_path,target_size=(224,224))  # this is a PIL image
        array = img_to_array(img)
        return array

In [50]:
metadata['img_path'] = '/mnt/images/' + metadata['image_id'].astype(str) + '.jpg'
metadata['img_array'] = metadata['img_path'].apply(lambda row: prepare_image(row))
metadata = metadata[pd.notnull(metadata['img_array'])]

x = np.asarray(metadata['img_array'].tolist()).reshape(len(metadata),3,224,224)
x.shape

(24624, 3, 224, 224)

In [51]:
def creating_labels(x):
    if ( 'topic' in x):
        return x['topic']
    elif ("nature" in str(x['tags_clean'])) or ("lake" in str(x['tags_clean'])) or ("river" in str(x['tags_clean'])) or ("view" in str(x['tags_clean'])) or ("beach" in str(x['tags_clean'])) or ("flowers" in str(x['tags_clean'])) or ("landscape" in str(x['tags_clean'])) or ("waterfall" in str(x['tags_clean'])) or ("sunrise" in str(x['tags_clean'])) or ("sunset" in str(x['tags_clean'])) or ("water" in str(x['tags_clean'])) or ("nationalpark" in str(x['tags_clean'])) or ("alaska" in str(x['tags_clean'])) or ("sky" in str(x['tags_clean'])) or ("yosemite" in str(x['tags_clean'])) or ("mountains" in str(x['tags_clean'])):
        return 'Natural Landscape'
    elif ("birds" in str(x['tags_clean'])) or ("wild" in str(x['tags_clean'])) or ("wildlife" in str(x['tags_clean'])) or ("forest" in str(x['tags_clean'])) or ("animals" in str(x['tags_clean'])) or ("zoo" in str(x['tags_clean'])):
        return 'Animals & Birds'
    elif ("food" in str(x['tags_clean'])) or ("brunch" in str(x['tags_clean'])) or ("dinner" in str(x['tags_clean'])) or ("lunch" in str(x['tags_clean'])) or ("bar" in str(x['tags_clean'])) or ("restaurant" in str(x['tags_clean'])) or ("drinking" in str(x['tags_clean'])) or ("eating" in str(x['tags_clean'])):
        return 'Food'
    elif ("urban" in str(x['tags_clean'])) or ("shop" in str(x['tags_clean'])) or ("market" in str(x['tags_clean'])) or ("square" in str(x['tags_clean'])) or ("building" in str(x['tags_clean'])) or ("citylights" in str(x['tags_clean'])) or ("cars" in str(x['tags_clean'])) or ("traffic" in str(x['tags_clean'])) or ("city" in str(x['tags_clean'])) or ("downtown" in str(x['tags_clean'])) or ("sanfrancisco" in str(x['tags_clean'])) or ("newyork" in str(x['tags_clean'])) or ("newyork" in str(x['tags_clean'])) or ("seattle" in str(x['tags_clean'])) or ("sandiego" in str(x['tags_clean'])) or ("washington" in str(x['tags_clean'])):
        return 'Urban Scenes'
    elif ("hotel" in str(x['tags_clean'])) or ("home" in str(x['tags_clean'])) or ("interior" in str(x['tags_clean'])):
        return 'Interiors'
    elif ("us" in str(x['tags_clean'])) or ("people" in str(x['tags_clean'])) or ("group" in str(x['tags_clean'])) or ("friends" in str(x['tags_clean'])):
        return 'people'
    else:
        return "Others"

In [52]:
metadata['tags_clean']= metadata['tags'].str.split()
metadata = metadata.replace(np.nan, '', regex=True)

metadata['labels'] = metadata.apply(creating_labels, axis=1)
topics = metadata['labels'].unique().tolist()
topics = list(set(topics) - set(['Others']))

metadata = metadata.loc[metadata['labels'].isin(topics)]
metadata['labels'].value_counts()

urban         6491
landscapes    5361
wildlife      4642
food          4615
people        2848
drinks         667
Name: labels, dtype: int64

In [65]:
label_map = d={x:i for i,x in enumerate(topics)}
y = metadata['labels'].apply(lambda row : label_map[row])
y.value_counts()

2    6491
0    5361
4    4642
5    4615
1    2848
3     667
Name: labels, dtype: int64

In [66]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

num_classes = 6
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 13)
# Transform targets to keras compatible format
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)


In [67]:
y_train.shape

(19699, 6)

In [72]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(6, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(x_train, y_train, nb_epoch=2, verbose = 1, validation_data = (x_test,y_test))

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/applications/inception_v3.py:367: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:24: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 19699 samples, validate on 4925 samples
Epoch 1/2
19699/19699 [==============================] - 242s 12ms/step - loss: 1.3007 - val_loss: 0.9852
Epoch 2/2
19699/19699 [==============================] - 235s 12ms/step - loss: 0.9523 - val_loss: 0.9207


In [115]:
#for i, layer in enumerate(base_model.layers):
    #print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(x_train, y_train, nb_epoch=30, verbose = 1, validation_data = (x_test,y_test))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 19699 samples, validate on 4925 samples
Epoch 1/30
19699/19699 [==============================] - 277s 14ms/step - loss: 0.5438 - acc: 0.8105 - val_loss: 0.8036 - val_acc: 0.7334
Epoch 2/30
19699/19699 [==============================] - 270s 14ms/step - loss: 0.5241 - acc: 0.8146 - val_loss: 0.8097 - val_acc: 0.7342
Epoch 3/30
19699/19699 [==============================] - 270s 14ms/step - loss: 0.4937 - acc: 0.8249 - val_loss: 0.8186 - val_acc: 0.7354
Epoch 4/30
19699/19699 [==============================] - 270s 14ms/step - loss: 0.4699 - acc: 0.8341 - val_loss: 0.8285 - val_acc: 0.7328
Epoch 5/30
19699/19699 [==============================] - 270s 14ms/step - loss: 0.4460 - acc: 0.8420 - val_loss: 0.8392 - val_acc: 0.7334
Epoch 6/30
19699/19699 [==============================] - 270s 14ms/step - loss: 0.4213 - acc: 0.8519 - val_loss: 0.8541 - val_acc: 0.7316
Epoch 7/30
19699/19699 [==============================] - 270s 14ms/step - loss: 0.3986 - acc: 0.8609 - val_loss: 0.8

In [75]:
model.save_weights('/mnt/cnn_2epoch_1210.1')

In [116]:
y_true, y_pred = y_test, model.predict(x_test)

In [117]:
y_true.shape, y_pred.shape

((4925, 6), (4925, 6))

ValueError: Mix type of y not allowed, got types {'multilabel-indicator', 'continuous-multioutput'}